In [47]:
from langchain_core.output_parsers import StrOutputParser
from PyPDF2 import PdfReader
from langchain_text_splitters  import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import SentenceTransformerEmbeddings
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from typing import List
import os
import getpass
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
api_key = os.getenv("GROQ_API_KEY")
api_key

'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'

## Load LLM

In [ ]:
def load_llm(id_model, temperature):
    llm = ChatGroq(
        model=id_model,
        temperature=temperature,
        api_key=api_key,
        max_tokens=1024,
        timeout=None,
        max_retries=3
    )
    return llm

llm = load_llm('meta-llama/llama-4-maverick-17b-128e-instruct', 0.3)


menggunakan model <b> meta-llama/llama-4-maverick-17b-128e-instruct </b> karena terbaru dan sangat mendukung integrasi Agent AI sehingga menghasilkan output yang akurat

In [50]:
# document laoder
def extract_text_from_pdf(file_path :str) -> List[Document]:
    try : 
        
        reader= PyMuPDFLoader(file_path)
        docs = reader.load()
        print(f"✅ Berhasil memproses Dokumen")
        return docs
    except Exception as e:
        print(f"Error saat memproses PDF : {str(e)}")
        return []
#chunking text
def chunk_text(documents : List[Document],chunk_size=1000,overlap=200) -> List[Document]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size= chunk_size,
        chunk_overlap=overlap,
        length_function=len,
        separators=["\n\n", "\n", '\n●','\n1','\n2','\n3','\n4','\n5','\n6','\n7','\n8','\n9','\n10'," "]
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Chunking selesai. Total halaman asli : {len(documents)}")
    return chunks

#index to qdrant
def index_to_qdrant(chunks : List[Document], url : str , collection_name : str)-> None:
    if not chunks:
        print("❌ Tidak ada chunks yang valid untuk di-index.")
        return

    try:
        # Inisialisasi model embedding yang akan digunakan
        model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

        print(f"⏳ Memulai indexing ke Qdrant Collection: {collection_name}...")
        
        # 2. Indexing Otomatis
        # Qdrant.from_documents secara otomatis menangani:
        # a) Mengubah chunks menjadi embeddings.
        # b) Menyimpan embeddings, teks, dan metadata ke Qdrant.
        Qdrant.from_documents(
            documents=chunks,
            embedding=model,
            url = url,
            collection_name=collection_name,
        )
        
        print("✅ Indexing ke Qdrant Selesai!")

    except Exception as e:
        print(f"❌ Gagal Indexing ke Qdrant. Cek koneksi atau API Key Anda. Error: {e}")



In [51]:
# pdf_path = 'Handbook_BNI_Mbank.pdf'
qdrant_url = "http://localhost:6333"
collection_name = 'user_manual_BNI_Mbank'

folder= "./static"
for filename in os.listdir(folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder,filename)   
        text_book = extract_text_from_pdf(pdf_path)
        chunk = chunk_text(text_book)

        #save qdrant
        index_to_qdrant(chunk,qdrant_url,collection_name)


✅ Berhasil memproses Dokumen
Chunking selesai. Total halaman asli : 31
⏳ Memulai indexing ke Qdrant Collection: user_manual_BNI_Mbank...
✅ Indexing ke Qdrant Selesai!


In [52]:
from qdrant_client import QdrantClient
from qdrant_client.http.exceptions import UnexpectedResponse

# Ganti 'localhost' dan 6333 dengan alamat server Qdrant Anda
QDRANT_HOST = "localhost"
QDRANT_PORT = 6333 # Port REST API

try:
    # Buat instance klien
    client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)
    print(f"Mencoba koneksi ke: {QDRANT_HOST}:{QDRANT_PORT}")
    
    # Tambahkan timeout untuk mencegah pemblokiran yang terlalu lama (opsional, default 5 detik)
    # client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT, timeout=5)

    # Lakukan operasi ringan yang memerlukan koneksi
    info = client.get_collections() 
    
    ## Alternatif: Cek informasi server (lebih mendalam)
    # info = client.get_locks() 
    # info = client.info()
    
    # Jika berhasil, cetak hasilnya
    print("✅ KONEKSI BERHASIL!")
    print(f"Informasi Koleksi Qdrant: {info.collections}")

except ConnectionRefusedError:
    print(f"❌ KONEKSI GAGAL: Server Qdrant tidak berjalan atau menolak koneksi pada {QDRANT_HOST}:{QDRANT_PORT}")
except UnexpectedResponse as e:
    # Terjadi jika server berjalan tetapi merespons dengan kode status error
    print(f"❌ KONEKSI GAGAL: Server merespons dengan Error: {e}")
except Exception as e:
    print(f"❌ KONEKSI GAGAL: Terjadi kesalahan lain: {e}")

Mencoba koneksi ke: localhost:6333
✅ KONEKSI BERHASIL!
Informasi Koleksi Qdrant: [CollectionDescription(name='book_knowledge'), CollectionDescription(name='user_manual_BNI_Mbank')]


## Semantic Search <br>

Menggunakan all-MiniLM-L6-v2 karena ringan dan cepat dibandingkan model embedding Indonesia

In [ ]:
# retriever
def get_qdrant_retriever(
    url: str = "http://localhost:6333",
    collection_name: str = None, 
    model_name : str = "all-MiniLM-L6-v2", 
    k : int = 4):
       
    embedding = SentenceTransformerEmbeddings(model_name=model_name)
    client = QdrantClient(url=url)
    print(f"Connect Ke Qdrant {url}")
    
    qdrant = Qdrant(
        client=client,
        collection_name=collection_name, # <== Penentuan COLLECTION
        embeddings=embedding,
    )
    retriever = qdrant.as_retriever(search_kwargs={"k" : k})
    return retriever

In [104]:
retriever = get_qdrant_retriever(collection_name=collection_name)

In [105]:
print(retriever.invoke('Fitur Whoosh')[0].page_content)

BNI Mobile Banking Handbook
09
Lifestyle: Tiket Whoosh
Kini hadir pemesanan Tiket Whoosh di fitur Lifestyle BNI Mobile 
Banking. Yuk berangkat!
Pilih Fitur Tiket Whoosh
Saatnya rasakan naik KCIC sekarang juga! Order langsung 
dari Fitur Lifestyle BNI Mobile Banking.
Ikuti setiap langkah pemesanan tiket Whoosh hingga 
pembayaran.
Kamu dapat melihat status pemesanan dan mendapatkan 
QR Code Tiket keberangkatanmu di BNI Mobile Banking.
Lakukan Pemesanan
hingga Pembayaran
Dapatkan Tiket
Keberangkatanmu
1
2
3
Tiket 
Whoosh


## RAG LANGCHAIN

In [70]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_classic.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableLambda, RunnableBranch, RunnablePassthrough


In [79]:
#prompt 
context_q_system_prompt = "Kamu adalah asisten AI yang membantu pengguna menjawab pertanyaan tentang BNI Mobile Banking. \
    Gunakan konteks dari dokumen (hasil pencarian retriever) dan riwayat percakapan sebelumnya \
    untuk menjawab dengan akurat, ringkas, dan relevan. \
    Jika konteks tidak mengandung informasi yang cukup untuk menjawab, katakan dengan sopan 'Maaf, saya tidak tahu.' Jangan menambahkan kata lagi"

context_q_user_prompt = "Pertanyaan: {input}\n\n Konteks:\n{context}"

context_q_prompt = ChatPromptTemplate.from_messages([
    ("system",context_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human",context_q_user_prompt)
])

In [84]:
#Mengecek apakah memiliki konteks
def has_context(inputs) : 
    context = inputs.get("context","")
    return bool(context and context.strip())


history_aware_retriever = create_history_aware_retriever(
    llm=llm,
    retriever=retriever,
    prompt=context_q_prompt
)



qa_chain = create_stuff_documents_chain(
    llm,
    context_q_prompt,
)

rag_chain = create_retrieval_chain(
    history_aware_retriever,
    qa_chain
)
no_context_chain = RunnableLambda(lambda x : {"answer" : "Maaf saya tidak tahu"})

final_chain = RunnableBranch(
    (has_context,rag_chain),
    RunnablePassthrough() | no_context_chain
)

## Penerapan AI Chat BOT

In [92]:
chat_history = []
input = "Sebutkan Fitur BNI Mobile?"
docs = retriever.invoke(input)
context = "\n\n".join([d.page_content for d in docs]) if docs else "Saya tidak tahu"

chat_history.append(HumanMessage(content=input))

In [93]:
result = rag_chain.invoke({"input":input, "chat_history":chat_history, "context" : context})
chat_history.append(AIMessage(content=result["answer"]))

In [94]:
print(result["answer"])

Fitur BNI Mobile Banking mencakup beberapa layanan, yaitu:

1. Pembayaran Pasca Bayar Telepon: memungkinkan pengguna membayar tagihan telepon setiap bulan.
2. Pembelian Voucher Prabayar Telepon: memungkinkan pengguna membeli berbagai voucher pulsa prabayar.
3. Pembelian Paket Data: memungkinkan pengguna membeli paket data melalui BNI Mobile Banking.

Fitur-fitur tersebut termasuk dalam kategori "Bayar & Beli Provider Telekomunikasi" di BNI Mobile Banking.


In [95]:
result2 = rag_chain.invoke({"input":"apa itu BRI Mobile", "chat_history":chat_history, "context" : context})
chat_history.append(AIMessage(content=result2["answer"]))

In [96]:
print(result2["answer"])

Maaf, saya tidak tahu apa itu BRI Mobile karena konteks yang diberikan hanya membahas tentang BNI Mobile Banking. BRI Mobile kemungkinan adalah aplikasi mobile banking dari Bank BRI, tetapi tidak ada informasi yang cukup dalam konteks yang diberikan untuk menjawab pertanyaan tersebut.


In [98]:
check_my_last_question = rag_chain.invoke({"input":"what my last question", "chat_history":chat_history,"context" : context})
print(check_my_last_question["answer"])

Pertanyaan terakhir Anda adalah "Sebutkan Fitur BNI Mobile?"


In [ ]:
input = "How to perform a transfer payment using BNI Mobile Banking"
chat_history.append(HumanMessage(content=input))

result2 = rag_chain.invoke({"input":input, "chat_history":chat_history})
chat_history.append(AIMessage(content=result2["answer"]))

print(result2["answer"])

To perform a transfer payment using BNI Mobile Banking via BI-Fast, follow these steps:

1. Click on the "Transfer Antar Bank" feature in the Transfer menu.
2. Input the data and select the transfer service type to BI-Fast.
3. Enter your transaction password to complete the transaction.

The transfer can be done 24/7, and the admin fee is Rp2,500.


In [ ]:
check_my_last_question = rag_chain.invoke({"input":"Apa pertanyaan terakhir saya", "chat_history":chat_history})
print(check_my_last_question["answer"])

Pertanyaan terakhir Anda adalah "How to perform a transfer payment using BNI Mobile Banking".


In [ ]:
chat_history

[HumanMessage(content='Apa saja fitur terbaru yang tersedia di BNI Mobile Banking?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The question "Apa saja fitur terbaru yang tersedia di BNI Mobile Banking?" can be rephrased as "What are the latest features available in BNI Mobile Banking?" without referencing the chat history. \n\nSince the provided context does not explicitly mention "latest features", a more accurate rephrased question would be "What features are available in BNI Mobile Banking?" \n\nThe context does provide some information about the features available in BNI Mobile Banking, such as payment and purchase of telecommunication providers, buying voucher pulsa prabayar, and paket data. However, it does not explicitly state that these are the "latest" features.\n\nTherefore, the rephrased question is: "What features are available in BNI Mobile Banking?"', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='berdasarkan pertanyaan terakhir,